In [48]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import scipy 
%matplotlib qt
from scipy import ndimage, misc

In [49]:
scipy.version.full_version

'1.7.1'

In [50]:
img = cv2.imread(r'C:\Users\fateme\Desktop\inputs\P3\sonderkommando_photographs_280.png') 
img_copy = np.copy(img)
img_copy = cv2.cvtColor(img_copy,cv2.COLOR_BGR2RGB)
plt.imshow(img_copy)

In [51]:
input_pts = np.float32([[133,134],[319,83],[120,515],[321,475]])
output_pts = np.float32([[97,80],[318,80],[97,478],[318,478]])
M = cv2.getPerspectiveTransform(input_pts,output_pts)
out = cv2.warpPerspective(img,M,(img.shape[1], img.shape[0]),flags=cv2.INTER_LINEAR)
plt.imshow(out)

In [52]:
out

array([[[ 17,  15,  15],
        [ 18,  16,  16],
        [ 18,  16,  16],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[ 15,  13,  13],
        [ 18,  16,  16],
        [ 20,  18,  18],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[ 15,  13,  13],
        [ 17,  15,  15],
        [ 19,  17,  17],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       ...,

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [240, 250, 250],
        [203, 212, 212],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [242, 252, 252],
        [196, 204, 204],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [247, 255, 255],
        [193, 199, 199],
        [  0,   0,   0]]

In [53]:
res = cv2.resize(out,None,fx=10, fy=10, interpolation = cv2.INTER_CUBIC)
plt.subplot(121),plt.imshow(img),plt.title('Input')
plt.subplot(122),plt.imshow(res),plt.title('Output')

(<AxesSubplot:title={'center':'Output'}>,
 Text(0.5, 1.0, 'Output'))

In [54]:
out.shape

(550, 419, 3)

In [55]:
rows,cols,r = out.shape
M = cv2.getRotationMatrix2D((cols/2,rows/2),-1,2)
dst = cv2.warpAffine(out,M,(cols,rows))
plt.subplot(121),plt.imshow(dst),plt.title('output')

(<AxesSubplot:title={'center':'output'}>,
 Text(0.5, 1.0, 'output'))

In [56]:
bilateral = cv2.bilateralFilter(dst, 10, 25, 25)
plt.subplot(121),plt.imshow(bilateral),plt.title('output')

(<AxesSubplot:title={'center':'output'}>,
 Text(0.5, 1.0, 'output'))

In [57]:
import scipy.misc
from scipy import misc
from scipy.misc.pilutil import Image
 
im_array = scipy.misc.fromimage(dst)
im_inverse = 255 - im_array
im_result = scipy.misc.toimage(im_inverse)
misc.imsave('result.tiff',im_result)

ModuleNotFoundError: No module named 'scipy.misc.pilutil'

In [ ]:
def guidedfilter(p, I, r, e, same_shape=True):

    if not(same_shape):
        I = np.repeat(I[:, :, np.newaxis], 3, axis=2)

    r = 2 * r - 1
    mean_I = cv2.blur(I,(r,r))
    mean_p = cv2.blur(p,(r,r))
    corr_I = cv2.blur(np.multiply(I,I),(r,r))
    corr_p = cv2.blur(np.multiply(I,p),(r,r))
    var_I = corr_I - np.multiply(mean_I,mean_I)
    cov_Ip = corr_p - np.multiply(mean_I,mean_p)
    a = np.divide(cov_Ip,(var_I+e))
    b = mean_p - np.multiply(a, mean_I)
    mean_a = cv2.blur(a,(r,r))
    mean_b = cv2.blur(b,(r,r))
    q = np.multiply(mean_a,I) + mean_b
    q = np.clip(q, 0, 1)
    return q

In [ ]:
img1 = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)/255
guided_img = guidedfilter(img1,img1,2,0.02)
plt.imshow(guided_img, cmap='gray')
plt.show()

In [ ]:
dst

In [ ]:
image = cv2.resize(dst, (2000, 2000))
image_bw = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
clahe = cv2.createCLAHE(clipLimit = 2)
final_img = clahe.apply(image_bw) + 10
_, ordinary_img = cv2.threshold(image_bw, 155, 255, cv2.THRESH_BINARY)
plt.figure(figsize=(12, 15))
plt.imshow(final_img,cmap="gray")

In [ ]:
def wgif(p, I, r, r2, e, same_shape=True):
    if not(same_shape):
        I = np.repeat(I[:, :, np.newaxis], 3, axis=2)
    r = 2 * r - 1
    r2 = 2 * r2 - 1
    mean_I = cv2.blur(I,(r,r))
    mean_I2 = cv2.blur(I,(r2,r2))
    mean_p = cv2.blur(p,(r,r))
    corr_I = cv2.blur(np.multiply(I,I),(r,r))
    corr_I2 = cv2.blur(np.multiply(I,I),(r2,r2))
    corr_p = cv2.blur(np.multiply(I,p),(r,r))
    var_I = corr_I - np.multiply(mean_I,mean_I)
    var_I2 = corr_I2 - np.multiply(mean_I2, mean_I2)
    cov_Ip = corr_p - np.multiply(mean_I,mean_p)
    landa = (0.001*1)**2
    si = (1 / (p.shape[0] * p.shape[1]) ) * (var_I2 + landa) * np.sum(np.reciprocal(var_I2+ landa))
    a = np.divide(cov_Ip,(var_I+e/si))
    b = mean_p - np.multiply(a, mean_I)
    mean_a = cv2.blur(a,(r,r))
    mean_b = cv2.blur(b,(r,r))
    q = np.multiply(mean_a,I) + mean_b
    return q


In [ ]:
def wgif(p, I, r, r2, e, same_shape=True):
    if not(same_shape):
        I = np.repeat(I[:, :, np.newaxis], 3, axis=2)
    r = 2 * r - 1
    r2 = 2 * r2 - 1
    mean_I = cv2.blur(I,(r,r))
    mean_I2 = cv2.blur(I,(r2,r2))
    mean_p = cv2.blur(p,(r,r))
    corr_I = cv2.blur(np.multiply(I,I),(r,r))
    corr_I2 = cv2.blur(np.multiply(I,I),(r2,r2))
    corr_p = cv2.blur(np.multiply(I,p),(r,r))
    var_I = corr_I - np.multiply(mean_I,mean_I)
    var_I2 = corr_I2 - np.multiply(mean_I2, mean_I2)
    cov_Ip = corr_p - np.multiply(mean_I,mean_p)
    landa = (0.001*1)**2
    si = (1 / (p.shape[0] * p.shape[1]) ) * (var_I2 + landa) * np.sum(np.reciprocal(var_I2+ landa))
    a = np.divide(cov_Ip,(var_I+e/si))
    b = mean_p - np.multiply(a, mean_I)
    mean_a = cv2.blur(a,(r,r))
    mean_b = cv2.blur(b,(r,r))
    q = np.multiply(mean_a,I) + mean_b
    return q


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from scipy import ndimage, misc

In [60]:
img = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
img = cv2.equalizeHist(img)/255
img = wgif(img, img,5,5, 0.1 ** 2)
plt.imshow(img,cmap="gray")
plt.show()

In [58]:
plt.imshow(img1,cmap="gray")

In [59]:
plt.imshow(img2,cmap="gray")

In [ ]:
img1.shape